In [1]:
# Importing the libraries
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from ast import literal_eval

In [3]:
data = pd.read_csv("C:/Users/hp/OneDrive/Desktop/NOTES/recommendationSys/Hotel_Reviews.csv")

In [4]:
data.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [5]:
# print rows and columns
data.shape

(515738, 17)

In [7]:
# check if any null values
data.isnull().sum()

Hotel_Address                                    0
Additional_Number_of_Scoring                     0
Review_Date                                      0
Average_Score                                    0
Hotel_Name                                       0
Reviewer_Nationality                             0
Negative_Review                                  0
Review_Total_Negative_Word_Counts                0
Total_Number_of_Reviews                          0
Positive_Review                                  0
Review_Total_Positive_Word_Counts                0
Total_Number_of_Reviews_Reviewer_Has_Given       0
Reviewer_Score                                   0
Tags                                             0
days_since_review                                0
lat                                           3268
lng                                           3268
dtype: int64

In [9]:
# Replace some country names
data.Hotel_Address = data.Hotel_Address.str.replace("United Kingdom", "UK")

In [10]:
# create a column countries from Hotel_Address
# Country name is in the last at every Address 
# apply - pass a function and apply it on every single value of the Pandas series.
data["Countries"] = data.Hotel_Address.apply(lambda x: x.split(" ")[-1])

In [11]:
data["Countries"]
# print(data.Countries.unique()) ['Netherlands' 'UK' 'France' 'Spain' 'Italy' 'Austria']

0         Netherlands
1         Netherlands
2         Netherlands
3         Netherlands
4         Netherlands
             ...     
515733        Austria
515734        Austria
515735        Austria
515736        Austria
515737        Austria
Name: Countries, Length: 515738, dtype: object

In [12]:
# Get columns
data.columns

Index(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
       'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'Tags',
       'days_since_review', 'lat', 'lng', 'Countries'],
      dtype='object')

In [13]:
# Drop unnecessary columns
changed_data = data.drop(['Additional_Number_of_Scoring',
       'Review_Date','Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score',
       'days_since_review', 'lat', 'lng'],axis=1)

In [14]:
# Modified dataframe
changed_data.head()

,Hotel_Address,Average_Score,Hotel_Name,Tags,Countries
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Duplex Double...",Netherlands
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Duplex Double...",Netherlands
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,"[' Leisure trip ', ' Family with young childre...",Netherlands
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",Netherlands
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",Netherlands


In [15]:
# convert Tags list to string
# apply - iterates over and applies to each content in Tag column
def changeToString(column):
    column = column[0]
    if (type(column) != list):
        return "".join(literal_eval(column))
    else:
        return column
    
changed_data["Tags"] = changed_data[["Tags"]].apply(changeToString, axis=1)

In [16]:
changed_data.head()

,Hotel_Address,Average_Score,Hotel_Name,Tags,Countries
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,Leisure trip Couple Duplex Double Room Sta...,Netherlands
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,Leisure trip Couple Duplex Double Room Sta...,Netherlands
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,Leisure trip Family with young children Dup...,Netherlands
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,Leisure trip Solo traveler Duplex Double Ro...,Netherlands
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,Leisure trip Couple Suite Stayed 2 nights ...,Netherlands


In [17]:
# convert to lowercase
changed_data['Countries'] = changed_data['Countries'].str.lower()
changed_data['Tags'] = changed_data['Tags'].str.lower()

In [18]:
changed_data[["Countries", "Tags"]]

,Countries,Tags
0,netherlands,leisure trip couple duplex double room sta...
1,netherlands,leisure trip couple duplex double room sta...
2,netherlands,leisure trip family with young children dup...
3,netherlands,leisure trip solo traveler duplex double ro...
4,netherlands,leisure trip couple suite stayed 2 nights ...
...,...,...
515733,austria,leisure trip family with older children 2 r...
515734,austria,leisure trip family with young children sta...
515735,austria,leisure trip family with older children 2 r...
515736,austria,leisure trip group standard triple room st...


In [19]:
# split description into tokens (words)
# remove stop words and lemmatize remaining
# set index of dataframe using np array till the number of row count
# then found cosine similarity between filtered_set and temp2_set store in cos[]
# place higher similarity scores ones higher (descending order)
def recommendHotel(location, description):
    description = description.lower()
    word_tokenize(description)
    stop_words = stopwords.words('english')
    lemm = WordNetLemmatizer()
    filtered  = {word for word in description if not word in stop_words}
    filtered_set = set()
    for fs in filtered:
        filtered_set.add(lemm.lemmatize(fs))

    country = changed_data[changed_data['Countries']==location.lower()]
    country = country.set_index(np.arange(country.shape[0]))
    cos = [];
    for i in range(country.shape[0]):
        temp_token = word_tokenize(country["Tags"][i])
        temp_set = [word for word in temp_token if not word in stop_words]
        temp2_set = set()
        for s in temp_set:
            temp2_set.add(lemm.lemmatize(s))
        vector = temp2_set.intersection(filtered_set)
        cos.append(len(vector))
    country['similarity']=cos
    country = country.sort_values(by='similarity', ascending=False)
    country.drop_duplicates(subset='Hotel_Name', keep='first', inplace=True)
    country.sort_values('Average_Score', ascending=False, inplace=True)
    country.reset_index(inplace=True)
    output = country[["Hotel_Name", "Average_Score", "Hotel_Address"]].head()
    return output

In [20]:
recommendHotel('Italy', 'I want to go on a business trip')

,Hotel_Name,Average_Score,Hotel_Address
0,Excelsior Hotel Gallia Luxury Collection Hotel,9.4,Piazza Duca D Aosta 9 Central Station 20124 Mi...
1,Palazzo Parigi Hotel Grand Spa Milano,9.3,Corso Di Porta Nuova 1 Milan City Center 20121...
2,Hotel Spadari Al Duomo,9.3,Via Spadari 11 Milan City Center 20123 Milan I...
3,Room Mate Giulia,9.3,Silvio Pellico 4 Milan City Center 20121 Milan...
4,UNA Maison Milano,9.3,Via Mazzini 4 Milan City Center 20123 Milan Italy


In [21]:
recommendHotel('Austria', 'I want to go on a family trip')

,Hotel_Name,Average_Score,Hotel_Address
0,Hotel Sacher Wien,9.5,Philharmoniker Stra e 4 01 Innere Stadt 1010 V...
1,Palais Coburg Residenz,9.5,Coburgbastei 4 01 Innere Stadt 1010 Vienna Aus...
2,Boutiquehotel Das Tyrol,9.4,Mariahilfer Stra e 15 06 Mariahilf 1060 Vienna...
3,Best Western Premier Kaiserhof Wien,9.4,Frankenberggasse 10 04 Wieden 1040 Vienna Austria
4,The Guesthouse Vienna,9.4,F hrichgasse 10 01 Innere Stadt 1010 Vienna Au...


In [22]:
import pickle
pickle.dump(changed_data, open('changed_df.pkl', 'wb'))